## LangChain - Agents Deep Dive

In [1]:
from langchain_core.tools import tool

In [2]:
@tool
def add(x: float, y: float) -> float:
    """
    Add 'x' and 'y'.
    """
    return x + y


@tool
def multiply(x: float, y: float) -> float:
    """
    Multiply 'x' and 'y'
    """
    return x * y


@tool
def exponentiate(x: float, y: float) -> float:
    """
    Raise 'x' to the power of 'y'
    """
    return x**y

@tool
def subtract(x: float, y: float) -> float:
    """
    Subtract 'x' from 'y'
    """
    return y - x

### Agent with LCEL (LangChain Expression Language)

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.base import RunnableSerializable
from langchain_openai import ChatOpenAI

In [6]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

In [4]:
prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user."
    )),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [8]:
tools = [add, subtract, multiply, exponentiate]

In [9]:
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")
)

In [10]:
tool_call = agent.invoke({"input": "What is 10 + 10?", "chat_history": []})
tool_call

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8e0Kwc2h8JVrrLfxRsJ2qX05', 'function': {'arguments': '{"x":10,"y":10}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 198, 'total_tokens': 215, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C3WiVBRb8vE1BNeejMWV3m7IAsNmS', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c8c89499-9094-43e2-ba62-049e70fa5a54-0', tool_calls=[{'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': 'call_8e0Kwc2h8JVrrLfxRsJ2qX05', 'type': 'tool_call'}], usage_metadata={'input_tokens': 198, 'output_tokens': 17, 'total_tokens': 215, 'input_token_details': {'audio': 0, 'cache_

In [11]:
tool_call.tool_calls

[{'name': 'add',
  'args': {'x': 10, 'y': 10},
  'id': 'call_8e0Kwc2h8JVrrLfxRsJ2qX05',
  'type': 'tool_call'}]

In [12]:
name2tool = {tool.name: tool.func for tool in tools}

In [13]:
name2tool

{'add': <function __main__.add(x: float, y: float) -> float>,
 'subtract': <function __main__.subtract(x: float, y: float) -> float>,
 'multiply': <function __main__.multiply(x: float, y: float) -> float>,
 'exponentiate': <function __main__.exponentiate(x: float, y: float) -> float>}

In [14]:
tool_exec_content = name2tool[tool_call.tool_calls[0]["name"]](**tool_call.tool_calls[0]["args"])
tool_exec_content

20

In [15]:
from langchain_core.messages import ToolMessage

In [16]:
tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_exec_content}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

In [ ]:
out = agent.invoke({
    "input": "What is 10+10?",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})
out
# note that the content is empty, thats because we defined the llm.bind_tools(tools, tool_choice="any")
# 'any' means that the agent will not answer directly, it will call a tool

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_AscZSjdURGPal0zAlpb6M7wf', 'function': {'arguments': '{"x":10,"y":10}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 227, 'total_tokens': 244, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C3X74U7wcaTMs34ekz42byGeYjYG2', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d1f74654-3fed-4ca7-b710-fbc96c516a35-0', tool_calls=[{'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': 'call_AscZSjdURGPal0zAlpb6M7wf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 227, 'output_tokens': 17, 'total_tokens': 244, 'input_token_details': {'audio': 0, 'cache_

In [20]:
# so we need to create a final_answer tool

@tool
def final_answer(answer: str, tool_used: list[str]) -> str:
    """
    Use this tool to provide a final answer to the user.
    The answer should be in natural language as this will be provided
    to the user directly. The tools_used must include a list of tool
    names that were used within the 'scratchpad'
    """
    return {"answer": answer, "tools_used": tool_used}

In [21]:
tools = [final_answer, add, subtract, multiply, exponentiate]

In [22]:
name2tool = {tool.name: tool.func for tool in tools}

In [23]:
agent2: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools=tools, tool_choice="any")
)

In [25]:
tool_call2 = agent2.invoke({"input": "What is 10+10?", "chat_history": []})
tool_call2.tool_calls

[{'name': 'add',
  'args': {'x': 10, 'y': 10},
  'id': 'call_kOevyv38N2zmeO1TGxCEkDta',
  'type': 'tool_call'}]

In [26]:
tool_out = name2tool[tool_call2.tool_calls[0]["name"]](**tool_call2.tool_calls[0]["args"])

tool_exec2 = ToolMessage(
    content=f"The {tool_call2.tool_calls[0]['name']} tool returned {tool_out}",
    tool_call_id=tool_call2.tool_calls[0]["id"]
)

out = agent2.invoke({
    "input": "What is 10+10?",
    "chat_history": [],
    "agent_scratchpad": [tool_call2, tool_exec2]
})
out

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_512hOB9FiPezbXOoIK9xR6W9', 'function': {'arguments': '{"answer":"10 + 10 equals 20.","tool_used":["functions.add"]}', 'name': 'final_answer'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 299, 'total_tokens': 326, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C3XJD3lW3QldryNmpCddXQNCa0RDw', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--01d3ceb2-2de8-4c2b-b986-04eadb1054fc-0', tool_calls=[{'name': 'final_answer', 'args': {'answer': '10 + 10 equals 20.', 'tool_used': ['functions.add']}, 'id': 'call_512hOB9FiPezbXOoIK9xR6W9', 'type': 'tool_call'}], usage_metada

In [27]:
out.tool_calls

[{'name': 'final_answer',
  'args': {'answer': '10 + 10 equals 20.', 'tool_used': ['functions.add']},
  'id': 'call_512hOB9FiPezbXOoIK9xR6W9',
  'type': 'tool_call'}]

In [28]:
out.tool_calls[0]["args"]

{'answer': '10 + 10 equals 20.', 'tool_used': ['functions.add']}